# TC4 - Lab exercises 2 

#####  Wafa Bouzouita et Dhia Eddine Yousfi

In this notebook, we are going to improve the POS tagger of last week. Instead of using a naive Bayes classifier, we will rely on a HMM where:
- the hidden states are POS tags
- the observations are words

It is a first order HMM where probabilities are defined as follows:
$$
p(y_1...y_n, x_1...x_n) = p(y_1) \prod_{i=2}^n p(y_i | y_{i-1}) \prod_{i=1}^n p(x_i | y_i)
$$

In [9]:
import nltk
import numpy as np
import sys
import math

# 1. Data

We first need to load and split the data between train and test data. You need to report the code from last week with the same split (90% train / 10% test).

In [10]:
# import dataset
data = nltk.corpus.brown.tagged_sents(tagset='universal')

In [11]:
from sklearn.model_selection import train_test_split
train_data, test_data =  train_test_split(data, test_size=0.1, random_state=42)

In [12]:
len(train_data), len(test_data)

(51606, 5734)

We will store the parameters of the HMM in numpy arrays. Therefore, to simplify the model, we rely on a dictionnary that maps words to tokens:
- the constructor takes as argument an iteratable over strings (e.g. list of strings) containing the vocabulary to store in the dictionnary
- you can set unk="\*UNK\*" to add entry for unknown strings (do not do it for POS tags!)
- len(dict) gives you the numbers of entry in the dict
- str_to_id maps a string to an index
- id_to_str gives you the string stored at a given index

In [13]:
class Dict:
    def __init__(self, words, unk=None):
        self._unk = unk
        self._word_to_id = dict()
        self._id_to_word = list()

        if unk in words:
            raise RuntimeError("UNK word exists in vocabulary")

        if unk is not None:
            self.unk_index = self._add_word(unk)

        for word in words:
            self._add_word(word)

    # for internal use only!
    def _add_word(self, word):
        if word not in self._word_to_id:
            id = len(self._id_to_word)
            self._word_to_id[word] = id
            self._id_to_word.append(word)
            return id
        else:
            return self._word_to_id[word]

    def str_to_id(self, word):
        if self._unk is not None:
            return self._word_to_id.get(word, self.unk_index)
        else:
            return self._word_to_id[word]

    def id_to_str(self, id):
        return self._id_to_word[id]

    def __len__(self):
        return len(self._word_to_id)

    def has_unk(self):
        return self._unk is not None
    
    def unk(self):
        return self.unk_index

Example:

In [14]:
test_dict = Dict(["a", "b", "c"], unk="*UNK*")
print("N. entry: ", len(test_dict))
print("Index of \"b\":", test_dict.str_to_id("b"))
# the following line does not throw an error because we gave a unk word
print("Index of \"e\":", test_dict.str_to_id("e"))
print(test_dict.__len__())

N. entry:  4
Index of "b": 2
Index of "e": 0
4


We now build the dictionnary of words and tags. We will restrict the dictionnary of words to contain only words that appears 10 or more times in the training data (use the code of last time).

For the dictionnary of words, the the unk parameters to any string you want. For the dictionnary of POS tags, do not set an unk word!

In [15]:
from collections import Counter
def compute_word_counts(data):
    dist_pos_tags =[]
    for j in range(len(data)):
        for i in range(len(data[j])):
            dist_pos_tags.append(data[j][i][0])
    return(dict(Counter(dist_pos_tags)))

In [16]:
train_word_counts = compute_word_counts(train_data)
word_dict = Dict([], unk="*UNK*")
tags_dict = Dict([])
for sentence in train_data:
    for word in sentence:
        if train_word_counts[word[0]] >= 10: 
            word_dict._add_word(word[0])
        tags_dict._add_word(word[1])

print(word_dict.__len__())

8284


# 2. Hidden Markov Model

The HMM class is a simple container for:
- the dictionnary of hidden states y_dict (i.e. dictionnary of tags)
- the dictionnary of observations x_dict (i.e. dictionnary of words)
- the parameters of the HMM:
    * init_prob $\in \mathbb R^{|Y|}$: initial tag probabilities $p(y_0) = init\_prob[y_0]$
    * transition_prob $\in \mathbb R^{|Y| \times |Y|}$: tag transition probabilities $p(y_i | y_{i - 1}) = transition\_prob[y_{i - 1}, y_i]$
    * observation_prob $\in \mathbb R^{|Y| \times |X|}$: observation probabilities $p(x_i | y_i) = observation\_prob[y_i, x_i]$

In [17]:
class HMM:
    def __init__(self, y_dict, x_dict):
        if not isinstance(y_dict, Dict) or not isinstance(x_dict, Dict):
            raise RuntimeError("Arguments must be of type Dict")

        self.y_dict = y_dict
        self.x_dict = x_dict

        n_y = len(y_dict)
        n_x = len(x_dict)
        self.init_prob = np.zeros((n_y,), float) 
        self.transition_prob = np.zeros((n_y, n_y), float) 
        self.observation_prob = np.zeros((n_y, n_x), float) 

## 2.1 Learning

Compute the matrices of probabilities hmm.init_prob, hmm.observation_prob and hmm.transition_prob from the data.

You **must** smooth the distributions!

In [18]:
hmm = HMM(tags_dict, word_dict)

init_word = []    


for sentence in train_data:
    hmm.init_prob[tags_dict.str_to_id(sentence[0][1])] +=1
    for i in range(len(sentence)):
        if i!=0:
            hmm.transition_prob[tags_dict.str_to_id(sentence[i-1][1])][tags_dict.str_to_id(sentence[i][1])] +=1
        if word_dict.str_to_id(sentence[i][0]):
            hmm.observation_prob[tags_dict.str_to_id(sentence[i][1])][word_dict.str_to_id(sentence[i][0])] +=1
        else:
            hmm.observation_prob[tags_dict.str_to_id(sentence[i][1])][0]


for i in range(tags_dict.__len__()):
    hmm.init_prob[i] = (hmm.init_prob[i] +1)/(tags_dict.__len__()+len(train_data))
    hmm.transition_prob[i] =(hmm.transition_prob[i]+1)/(tags_dict.__len__()+hmm.transition_prob[i].sum())  
    hmm.observation_prob[i] =(hmm.observation_prob[i]+1)/(word_dict.__len__()+hmm.observation_prob[i].sum())            

The following three cells check that the distribution you computed correctly sum to one. The first cell should output 1.0, the two others should output array containing twelve times the number 1.

In [19]:
hmm.init_prob.sum()

1.0

In [20]:
hmm.observation_prob.sum(1)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [21]:
hmm.transition_prob.sum(1)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

# 2.2. Viterbi

Implement the viterbi **without** computing in the log domain. What tagging accuracy do you achieve? How is it compared to the naive Bayes model of last week?

In [23]:
def viterbi(hmm, words):
    """
    Input:
    - hmm: an HMM object
    - words: a list of words (ie a sentence)
    Return:
    - a list of POS tags
    """
    nx = len(words)
    ny = hmm.y_dict.__len__()
    
    chart = np.zeros((ny, nx), float)
    
    backptr = np.zeros((ny, nx), int)
    
    # compute p(y1) * p(x2/y1)
    for i in range(ny):
        chart[i,0] =  hmm.observation_prob[i,hmm.x_dict.str_to_id(words[0])]*hmm.init_prob[i]
   
    for j in range(1,nx):
            for i in range(ny):
                mat = hmm.transition_prob[:, i] * hmm.observation_prob[i, hmm.x_dict.str_to_id(words[j])] * chart[:,j-1]                                                                                        
                chart[i,j] = max(mat)
                backptr[i,j] = mat.argmax()
    
    
    y = np.zeros((nx,))
    
    y[nx-1] = int(chart[:,nx-1].argmax())
    for j in range(nx-2, -1, -1):
        y[j] = backptr[int(y[j+1]),j+1]
    
    y = [hmm.y_dict.id_to_str(int(i)) for i in y]
    
    return y



In [24]:
# Evaluate the HMM using the viterbi
n_tags = 0
n_correct_tags = 0
for sentence in test_data:
    words = [w for w, t in sentence]
    pred = viterbi(hmm, words)
    n_tags += len(sentence)
    n_correct_tags += sum(1 for w in range(len(sentence))  if sentence[w][1] == pred[w])

print("Tagging accuract: %.2f" % (100 * n_correct_tags / n_tags))

Tagging accuract: 90.06


# 2.3. Viterbi in the log domain

Copy/paste you code from the previous cell and change it to compute in the log domain. What tagging accuracy do you achieve? How is it compared to the naive Bayes model of last week and to the previous implementation of the viterbi?

In [25]:
def viterbi_log(hmm, words):
    """
    Input:
    - hmm: an HMM object
    - words: a list of words (ie a sentence)
    Return:
    - a list of POS tags
    """
    eps = 1e-3
    n = len(words)
    chart = np.zeros((hmm.y_dict.__len__(), len(words)), float)
    backptr = np.zeros((hmm.y_dict.__len__(), len(words)), int)
    for i in range(hmm.y_dict.__len__()):
        chart[i,0] =  np.log(hmm.observation_prob[i,hmm.x_dict.str_to_id(words[0])]+eps)+np.log(hmm.init_prob[i])
    for j in range(1,len(words)):
            for i in range(hmm.y_dict.__len__()):
                mat =  np.log(hmm.transition_prob[:, i]+eps)+  np.log(hmm.observation_prob[i, hmm.x_dict.str_to_id(words[j])]+eps) + np.log(chart[:,j-1]+eps)                                                                                        
                chart[i,j] = max(mat)
                backptr[i,j] = mat.argmax()
                
    y = np.zeros((n,))
    y[n-1] = int(chart[:,n-1].argmax())
    for j in range(n-2, -1, -1):
        y[j] = backptr[int(y[j+1]),j+1]
    
    y = [hmm.y_dict.id_to_str(int(i)) for i in y]

    return y


In [26]:
# Evaluate the HMM using the viterbi in the log domain

n_tags = 0
n_correct_tags = 0
for sentence in test_data:
    words = [w for w, t in sentence]
    pred = viterbi_log(hmm, words)
    n_tags += len(sentence)
    n_correct_tags += sum(1 for w in range(len(sentence))  if sentence[w][1] == pred[w])

print("Tagging accuract: %.2f" % (100 * n_correct_tags / n_tags))

/home/wafe/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in log


Tagging accuract: 11.91


# 3. Marginalization

As a last exercise, implement function that evaluate the probability of a sequence of words and a sequence of hidden states given a HMM.

In [27]:
def probabilit_y(hmm, tags):
    
    p= hmm.init_prob[hmm.y_dict.str_to_id(tags[0])]
    
    # compute p(y1) * p(y2/y1) * .... p(yn/yn-1)
    for i in range(len(tags)):
        p *= hmm.transition_prob[hmm.y_dict.str_to_id(tags[i-1]),hmm.y_dict.str_to_id(tags[i])]
    return(p)
        

In [28]:
import random 
tags = "DET NOUN VERB DET ADJ NOUN .".split()
print(probabilit_y(hmm, tags))
random.shuffle(tags)
print(probabilit_y(hmm, tags))

1.6840173548136245e-05
1.3517657526672398e-08


In [29]:
def probabilit_x(hmm, words):

    nx = len(words)  
    ny = hmm.y_dict.__len__()
    
    chart = np.zeros((ny, nx), float)
    
    for i in range(ny):
        chart[i,0] =  hmm.observation_prob[i,hmm.x_dict.str_to_id(words[0])]*hmm.init_prob[i]
    
    for j in range(1,nx):
            for i in range(ny):
                mat = hmm.transition_prob[:, i] * hmm.observation_prob[i, hmm.x_dict.str_to_id(words[j])] * chart[:,j-1]                                                                                        
                chart[i,j] = mat.sum()  
    return(chart[:,nx-1].sum())
    

In [30]:
sentence = "This is a sentence .".split()
print(probabilit_x(hmm, sentence))
random.shuffle(sentence)
print(probabilit_x(hmm, sentence))

1.4859968613777992e-12
1.2016005704623697e-15
